In [4]:
import requests
import json
import time
import sys

# 👇 100% 정확한 키
API_ID = '692f7afdc2302605ddb0b5fb'
API_KEY = '7974a64c-a29d-4ef2-8a31-3ddcaec648d5'

def fetch_data_final():
    print("🚀 [Final Attempt] 데이터 수집 시작...")

    # 1. 인증 우회 전략 (쿼리 파라미터 + 헤더 동시 사용)
    url = "https://api.techspecs.io/v4/product/search"
    
    # ★ 핵심: 헤더와 쿼리 파라미터 양쪽에 다 넣어서 하나라도 걸리게 함
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "x-api-key": API_KEY,  # 헤더 방식
        "x-api-id": API_ID
    }
    
    params = {
        "query": "smartphone",
        "page": 1,
        "limit": 20,
        "api_key": API_KEY,    # 쿼리 파라미터 방식 (구버전 호환)
        "api_id": API_ID
    }

    try:
        print(f"👉 검색 요청 중: {url}")
        # verify=False는 SSL 에러 방지용 (개발용으로만 사용)
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 401:
            print("\n🚨 401 인증 에러 지속됨. 원인 분석:")
            print("1. API Key가 'Bearer Token'용이 아님 (확인됨)")
            print("2. 무료 한도 초과 가능성 (401로 표시될 수 있음)")
            print("3. 이 키는 'x-api-key' 전용이나, v4 엔드포인트가 일시적으로 막힘")
            print("\n⚠️ 긴급 처방: v3 엔드포인트(구버전) 시도...")
            
            # v3 시도 (구버전은 보통 x-api-key 잘 먹음)
            url_v3 = "https://api.techspecs.io/v3/product/search"
            response = requests.get(url_v3, headers=headers, params=params)

        if response.status_code != 200:
            print(f"❌ 실패! Status: {response.status_code}")
            print(f"   응답: {response.text}")
            return

        # 성공 시 데이터 처리
        data = response.json()
        items = []
        # 데이터 구조 유연하게 찾기
        if "data" in data and "items" in data["data"]: items = data["data"]["items"]
        elif "data" in data and isinstance(data["data"], list): items = data["data"]
        elif "items" in data: items = data["items"]
        elif "results" in data: items = data["results"]
            
        print(f"📦 {len(items)}개 제품 발견. 상세 조회 시작...")

        final_data = []
        for item in items:
            p_id = item.get("id", item.get("_id"))
            
            # 상세 조회
            detail_url = f"https://api.techspecs.io/v4/product/{p_id}"
            full_data = item
            try:
                d_res = requests.get(detail_url, headers=headers, params={"api_key": API_KEY, "api_id": API_ID})
                if d_res.status_code == 200:
                    d_json = d_res.json()
                    full_data = d_json.get("data", d_json)
            except: pass

            # 이미지 추출
            image_url = ""
            img_obj = full_data.get("image")
            if isinstance(img_obj, str): image_url = img_obj
            elif isinstance(img_obj, dict): 
                image_url = img_obj.get("front") or img_obj.get("back")
            
            if not image_url and full_data.get("images"):
                image_url = full_data["images"][0]
            if not image_url:
                image_url = "https://placehold.co/300x400?text=No+Image"

            clean_item = {
                "id": full_data.get("id"),
                "name": full_data.get("name", "Unknown"),
                "brand": full_data.get("brand", "Unknown"),
                "category": full_data.get("category", "Smartphone"),
                "price": full_data.get("price", "N/A"),
                "thumbnail": image_url,
                "specs": {
                    "cpu": full_data.get("hardware", {}).get("cpu"),
                    "ram": full_data.get("hardware", {}).get("ram"),
                    "storage": full_data.get("hardware", {}).get("storage"),
                    "display": full_data.get("display", {}).get("size"),
                    "battery": full_data.get("battery", {}).get("capacity")
                },
                "raw": full_data
            }
            final_data.append(clean_item)
            sys.stdout.write(".")
            sys.stdout.flush()
            time.sleep(0.2)

        with open("all-products.json", "w", encoding="utf-8") as f:
            json.dump(final_data, f, indent=2, ensure_ascii=False)
        print(f"\n\n💾 저장 완료! all-products.json (총 {len(final_data)}개)")

    except Exception as e:
        print(f"\n❌ 에러: {e}")

if __name__ == "__main__":
    fetch_data_final()


🚀 [Final Attempt] 데이터 수집 시작...
👉 검색 요청 중: https://api.techspecs.io/v4/product/search

🚨 401 인증 에러 지속됨. 원인 분석:
1. API Key가 'Bearer Token'용이 아님 (확인됨)
2. 무료 한도 초과 가능성 (401로 표시될 수 있음)
3. 이 키는 'x-api-key' 전용이나, v4 엔드포인트가 일시적으로 막힘

⚠️ 긴급 처방: v3 엔드포인트(구버전) 시도...
❌ 실패! Status: 404
   응답: <html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>

